[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intel/e2eAIOK/blob/main/demo/builtin/rnnt/RNNT_DEMO.ipynb)

# RNN-T Demo

Automatic speech recognition (ASR) systems convert audio into text representation. RNN-T is an end-to-end rnn based ASR model that directly output word transcripts given the input audio. This notebook contains step by step guide on how to optimize RNN-T model with Intel® End-to-End AI Optimization Kit, and detailed performance analysis.

# Content
* [Model Architecture](#Model-Architecture)
* [Optimizations](#Optimizations)
* [DEMO](#DEMO)

## ASR
<img src="./img/asr.png" width="800"/>

* The traditional ASR system (top picture) contains acoustic, phonetic and language components that work together as in a pipeline system
* The end-to-end ASR system is a single neural network that receives raw audio signal as input and provides a sequence of words at output

## Model Architecture
<img src="./img/rnnt_structure.png"/>

RNN-T is an end-to-end ASR model that directly converts audio into text representation.

The encoder network is a RNN which maps input acoustic frames into a higher-level representation.
The prediction network is a RNN that is explicitly conditioned on the history of previous non-blank targets predicted by the model.
The joint network is a feed-forward network that combines the outputs of the prediction network and the encoder to produce logits followed by a softmax layer to produce a distribution over the next output symbol.

## Optimizations

### Model architecture Intro

For RNN-T model democratization, we enabled distributed training with pytorch DDP to scale out model training on multi nodes, added time stack layer and increased time stack factor to reduce input sequence lengh, added layer and batch normalization to speedup training converge, decreased layer size to get a lighter model.

<img src="./img/model_base.png" width="600"/><figure>base model</figure>
<img src="./img/model_opt.png" width="600"/><figure>democratized model</figure>


### Distributed training

``` python
# data parallel
if world_size > 1:
    model = DDP(model, find_unused_parameters=True)
```

### Add time stack layer

For ASR systems, the number of time frames for an audio input sequence is significantly higher than the number of output text labels. LSTM is sequential model which leads to much time cost in process long sequence data like audio data. The StackTime layer stacks audio frames to reduce sequence length and form a higher dimension input, which helps to speedup training process.

```python
class StackTime(nn.Module):
    def __init__(self, factor):
        super().__init__()
        self.factor = int(factor)

    def stack(self, x):
        x = x.transpose(0, 1)
        T = x.size(1)
        padded = torch.nn.functional.pad(x, (0, 0, 0, (self.factor - (T % self.factor)) % self.factor))
        B, T, H = padded.size()
        x = padded.reshape(B, T // self.factor, -1)
        x = x.transpose(0, 1)
        return x

    def forward(self, x, x_lens):
        if type(x) is not list:
            x = self.stack(x)
            x_lens = (x_lens.int() + self.factor - 1) // self.factor
            return x, x_lens
        else:
            if len(x) != 2:
                raise NotImplementedError("Only number of seq segments equal to 2 is supported")
            assert x[0].size(1) % self.factor == 0, "The length of the 1st seq segment should be multiple of stack factor"
            y0 = self.stack(x[0])
            y1 = self.stack(x[1])
            x_lens = (x_lens.int() + self.factor - 1) // self.factor
            return [y0, y1], x_lens
```

About 4x speedup after increase time stack factor from 2 to 8.

<img src="./img/time_stack_2.PNG" width="600"/><figure>time_stack = 2</figure>
<img src="./img/time_stack_8.PNG" width="600"/><figure>time_stack = 8</figure>

Profiling data proves that less time cost on forward/backward since input sequence reduced with time stack layer

<img src="./img/stack_profile_base.png" width="600"/><figure>base model profiling</figure>
<img src="./img/stack_profile_democratize.png" width="600"/><figure>democratized model profiling</figure>


## Add layer normalization and batch normalization

Layer normalization for LSTM is important to the success of RNN-T modeling. Add layer normalization for LSTM and batch normalization for input feature help to speedup training converge. It takes 52 epochs to converge without normalization, while only 49 epochs needed with normalization. 

```python
enc_mod["batch_norm"] = nn.BatchNorm1d(pre_rnn_input_size)
```

```python
self.layer_norm = torch.nn.LayerNorm(hidden_size)
```

<img src="./img/no_norm.PNG" width="600"/><figure>without normalization</figure>
<img src="./img/norm.PNG" width="600"/><figure>with normalization</figure>


## HPO with SDA (Smart Democratization Advisor)

SDA config

```
Parameters for SDA auto optimization:
- learning_rate: 1.0e-3~1.0e-2 #training learning rate
- warmup_epochs: 1~10 #epoch to warmup learning rate
metrics:
- name: training_time # training time threshold
  objective: minimize
  threshold: 43200
- name: WER # training metric threshold
  objective: minimize
  threshold: 0.25
 ```

request suggestions from SDA

```python
suggestion = self.conn.experiments(self.experiment.id).suggestions().create()
```


## Framework related optimization

leverage IPEX for distributed training and enable socket binding for training in two socket system

```bash
# Use IPEX launch to launch training, enable NUMA binding in two socket system.
${CONDA_PREFIX}/bin/python -m intel_extension_for_pytorch.cpu.launch --distributed --nproc_per_node=2 --nnodes=4 --hostfile hosts train.py ${ARGS}
```

<img src="./img/no_numa_binding.png" width="600"/><figure>without numa binding</figure>
<img src="./img/numa_binding.png" width="600"/><figure>enable numa binding</figure>


# DEMO
* [Environment Setup](#Environment-setup)
* [Launch training](#Launch-training)

## Environment setup

### Option1 Setup Environment with Docker
``` bash
# Setup ENV
git clone https://github.com/intel/e2eAIOK.git
cd e2eAIOK
git submodule update --init --recursive
python3 scripts/start_e2eaiok_docker.py -b pytorch110 -w ${host0} ${host1} ${host2} ${host3} --proxy ""
# Enter Docker
sshpass -p docker ssh ${host0} -p 12345
```

### Option2 Setup Environment with Pip
pre-work: move e2eAIOK source code to /home/vmagent/app/e2eaiok

In [1]:
%%bash
pip install torchaudio==0.12.1 torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cpu
pip install oneccl_bind_pt==1.12.100 intel-extension-for-pytorch==1.12.100 -f https://developer.intel.com/ipex-whl-stable
pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110==1.9.0
pip install git+https://github.com/NVIDIA/dllogger#egg=dllogger
pip install "git+https://github.com/mlperf/logging.git@1.0.0"
pip install sentencepiece unidecode tensorboard inflect soundfile librosa sox pandas
git clone https://github.com/HawkAaron/warp-transducer && cd warp-transducer \
    && mkdir build && cd build \
    && cmake .. && make && cd ../pytorch_binding \
    && python setup.py install
pip install e2eAIOK-sda --pre
apt install -y numactl

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu


  Using cached https://download.pytorch.org/whl/cpu/torchaudio-0.12.1%2Bcpu-cp39-cp39-linux_x86_64.whl (3.5 MB)
  Using cached https://download.pytorch.org/whl/cpu/torch-1.12.1%2Bcpu-cp39-cp39-linux_x86_64.whl (189.2 MB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)


Looking in links: https://developer.intel.com/ipex-whl-stable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 17.1 MB/s eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist
  Using cached https://developer.download.nvidia.cn/compute/redist/nvidia-dali-cuda110/nvidia_dali_cuda110-1.9.0-3647997-py3-none-manylinux2014_x86_64.whl (680.9 MB)


  Cloning https://github.com/NVIDIA/dllogger to /tmp/pip-install-usu5pi_7/dllogger_95e707ff269b484eb37c3a9d466b7903


...

/usr/bin/ld: warning: /opt/intel/oneapi/intelpython/latest/envs/pytorch/lib/libgcc_s.so.1: unsupported GNU_PROPERTY_TYPE (5) type: 0xc0010001
/usr/bin/ld: warning: /opt/intel/oneapi/intelpython/latest/envs/pytorch/lib/libgcc_s.so.1: unsupported GNU_PROPERTY_TYPE (5) type: 0xc0010002
/usr/bin/ld: warning: /opt/intel/oneapi/intelpython/latest/envs/pytorch/lib/libgcc_s.so.1: unsupported GNU_PROPERTY_TYPE (5) type: 0xc0010001
/usr/bin/ld: warning: /opt/intel/oneapi/intelpython/latest/envs/pytorch/lib/libgcc_s.so.1: unsupported GNU_PROPERTY_TYPE (5) type: 0xc0010002


creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/warprnnt_pytorch
copying build/lib.linux-x86_64-3.9/warprnnt_pytorch/__init__.py -> build/bdist.linux-x86_64/egg/warprnnt_pytorch
copying build/lib.linux-x86_64-3.9/warprnnt_pytorch/warp_rnnt.cpython-39-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/egg/warprnnt_pytorch
byte-compiling build/bdist.linux-x86_64/egg/warprnnt_pytorch/__init__.py to __init__.cpython-39.pyc
creating stub loader for warprnnt_pytorch/warp_rnnt.cpython-39-x86_64-linux-gnu.so
byte-compiling build/bdist.linux-x86_64/egg/warprnnt_pytorch/warp_rnnt.py to warp_rnnt.cpython-39.pyc
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying warprnnt_pytorch.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying warprnnt_pytorch.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying warprnnt_pytorch.egg-info/dependency_links.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying warprnnt_py

zip_safe flag not set; analyzing archive contents...
warprnnt_pytorch.__pycache__.warp_rnnt.cpython-39: module references __file__


creating dist
creating 'dist/warprnnt_pytorch-0.1-py3.9-linux-x86_64.egg' and adding 'build/bdist.linux-x86_64/egg' to it
removing 'build/bdist.linux-x86_64/egg' (and everything under it)
Processing warprnnt_pytorch-0.1-py3.9-linux-x86_64.egg
creating /opt/intel/oneapi/intelpython/python3.9/envs/pytorch/lib/python3.9/site-packages/warprnnt_pytorch-0.1-py3.9-linux-x86_64.egg
Extracting warprnnt_pytorch-0.1-py3.9-linux-x86_64.egg to /opt/intel/oneapi/intelpython/python3.9/envs/pytorch/lib/python3.9/site-packages
Adding warprnnt-pytorch 0.1 to easy-install.pth file

Installed /opt/intel/oneapi/intelpython/python3.9/envs/pytorch/lib/python3.9/site-packages/warprnnt_pytorch-0.1-py3.9-linux-x86_64.egg
Processing dependencies for warprnnt-pytorch==0.1
Finished processing dependencies for warprnnt-pytorch==0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 214.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

Reading package lists...
Building dependency tree...
Reading state information...
numactl is already the newest version (2.0.11-2.1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


## Workflow Prepare

``` bash
# prepare model codes
cd /home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch
bash patch_rnnt.sh

# Download Dataset
# Download and unzip dataset from https://www.openslr.org/12 to /home/vmagent/app/dataset/LibriSpeech

# Generate tokenizer and tokenize text
cd /home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch
bash scripts/preprocess_librispeech.sh
```

Notes: RNN-T training is based on LibriSpeech train-clean-100 and evaluated on dev-clean, we evaluated WER with stock model (based on MLPerf submission) at train-clean-100 dataset, and final WER is 0.25, all the following optimization guarantee 0.25 WER. MLPerf submission took 38.7min with 8x A100 on LibriSpeech train-960h dataset.

public reference on train-clean-100: https://arxiv.org/pdf/1807.10893.pdf, https://arxiv.org/pdf/1811.00787.pdf

## Launch training

edit conf/e2eaiok_defaults_rnnt_example.conf

```
### GLOBAL SETTINGS ###
observation_budget: 1
save_path: /home/vmagent/app/e2eaiok/result/
ppn: 2
train_batch_size: 8
eval_batch_size: 8
iface: lo
hosts:
- localhost
epochs: 2
```

In [10]:
%%bash
cd /home/vmagent/app/e2eaiok
# sed -i '/ppn:/ s/:.*/: 1/' tests/cicd/conf/e2eaiok_defaults_rnnt_example.conf
python run_e2eaiok.py --data_path /home/vmagent/app/dataset/LibriSpeech --model_name rnnt --conf tests/cicd/conf/e2eaiok_defaults_rnnt_example.conf 

2023-03-20 02:41:47,824 - E2EAIOK - INFO - Above info is history record of this model
2023-03-20 02:41:47,824 - E2EAIOK.SDA - INFO - ### Ready to submit current task  ###
2023-03-20 02:41:47,825 - E2EAIOK.SDA - INFO - Model Advisor created
2023-03-20 02:41:47,825 - E2EAIOK.SDA - INFO - model parameter initialized
2023-03-20 02:41:47,825 - E2EAIOK.SDA - INFO - start to launch training
2023-03-20 02:41:47,825 - sigopt - INFO - training launch command: /opt/intel/oneapi/intelpython/latest/envs/pytorch/bin/python -m intel_extension_for_pytorch.cpu.launch --distributed --nproc_per_node=2 --nnodes=1 --hostfile hosts /home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/train.py --output_dir /home/vmagent/app/e2eaiok/result/357dc3f8a3dfe894b3a3fcdd15fd1129f95f71cf887c8475679b1ff5b50674d8 --dist --dist_backend gloo --batch_size 8 --val_batch_size 8 --lr 0.007 --warmup_epochs 6 --beta1 0.9 --beta2 0.999 --max_duration 16.7 --target 0.25 --min_lr 1e-05 --lr_exp_gamma 0.939 --epochs 2 --epochs_this_job

[0] No module named 'torch_ccl'
[1] No module named 'torch_ccl'
[0] world_size:2,rank:0
[1] world_size:2,rank:1


[0] 2023-03-20 02:41:49,890 - torch.distributed.distributed_c10d - INFO - Added key: store_based_barrier_key:1 to store for rank: 0
[1] 2023-03-20 02:41:49,890 - torch.distributed.distributed_c10d - INFO - Added key: store_based_barrier_key:1 to store for rank: 1
[0] 2023-03-20 02:41:49,890 - torch.distributed.distributed_c10d - INFO - Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 2 nodes.
[1] 2023-03-20 02:41:49,890 - torch.distributed.distributed_c10d - INFO - Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 2 nodes.


[0] :::MLLOG {"namespace": "", "time_ms": 1679280109904, "event_type": "INTERVAL_START", "key": "init_start", "value": null, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/train.py", "lineno": 357}}
[0] :::MLLOG {"namespace": "", "time_ms": 1679280110003, "event_type": "POINT_IN_TIME", "key": "seed", "value": 2021, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/train.py", "lineno": 362}}
[0] DLL 2023-03-20 02:41:50.006883 - PARAMETER | epochs :  2
[0] DLL 2023-03-20 02:41:50.006950 - PARAMETER | warmup_epochs :  6
[0] DLL 2023-03-20 02:41:50.007040 - PARAMETER | hold_epochs :  40
[0] DLL 2023-03-20 02:41:50.007164 - PARAMETER | epochs_this_job :  0
[0] DLL 2023-03-20 02:41:50.007195 - PARAMETER | cudnn_benchmark :  True
[0] DLL 2023-03-20 02:41:50.007238 - PARAMETER | amp_level :  1
[0] DLL 2023-03-20 02:41:50.007282 - PARAMETER | seed :  2021
[0] DLL 2023-03-20 02:41:50.007326 - PARAMETER | local_rank :  0
[0] DLL 2023-03-20 02:41:50.00737

[0] :::MLLOG {"namespace": "", "time_ms": 1679280110031, "event_type": "POINT_IN_TIME", "key": "model_weights_initialization_scale", "value": 0.5, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/train.py", "lineno": 397}}
[0] :::MLLOG {"namespace": "", "time_ms": 1679280110115, "event_type": "POINT_IN_TIME", "key": "weights_initialization", "value": null, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/common/rnn.py", "lineno": 89, "tensor": "pre_rnn"}}
[0] :::MLLOG {"namespace": "", "time_ms": 1679280110371, "event_type": "POINT_IN_TIME", "key": "weights_initialization", "value": null, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/common/rnn.py", "lineno": 89, "tensor": "post_rnn"}}
[0] :::MLLOG {"namespace": "", "time_ms": 1679280110375, "event_type": "POINT_IN_TIME", "key": "weights_initialization", "value": null, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/rnnt/model.py", "lineno": 159, "t

...

[0] :::MLLOG {"namespace": "", "time_ms": 1679280146604, "event_type": "POINT_IN_TIME", "key": "eval_accuracy", "value": 20.484347826086957, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/train.py", "lineno": 260, "epoch_num": 1}}
[0] :::MLLOG {"namespace": "", "time_ms": 1679280146604, "event_type": "INTERVAL_END", "key": "eval_stop", "value": null, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/train.py", "lineno": 261, "epoch_num": 1}}
[0] DLL 2023-03-20 02:42:26.605304 - epoch    1 |   dev ema wer 2048.43 | took 11.88 s
[0] :::MLLOG {"namespace": "", "time_ms": 1679280146605, "event_type": "INTERVAL_END", "key": "block_stop", "value": null, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo/rnnt/pytorch/train.py", "lineno": 811, "first_epoch_num": 1}}
[0] :::MLLOG {"namespace": "", "time_ms": 1679280146605, "event_type": "INTERVAL_START", "key": "block_start", "value": null, "metadata": {"file": "/home/vmagent/app/e2eaiok/modelzoo

[0]                                            aten::narrow         0.35%      79.656ms         1.00%     225.783ms       1.700us        132830  
[0]                                               aten::div         0.32%      71.734ms         0.32%      72.809ms     271.675us           268  
[0]                                            aten::expand         0.28%      62.077ms         0.40%      90.029ms       1.173us         76774  
[0]                                  aten::sigmoid_backward         0.27%      61.088ms         0.27%      61.088ms       6.646us          9192  
[0]                                   aten::constant_pad_nd         0.25%      57.246ms         0.84%     190.251ms      16.336us         11646  
[0]     autograd::engine::evaluate_function: AddmmBackward0         0.24%      53.343ms        16.44%        3.702s       1.203ms          3078  
[0]                                              aten::mul_         0.23%      52.620ms         0.26%      57.917ms     304.

[0] autograd::engine::evaluate_function: torch::autograd...         0.01%       2.367ms         0.20%      44.614ms     719.581us            62  
[0]     autograd::engine::evaluate_function: StackBackward0         0.01%       2.090ms         0.04%       8.795ms     314.107us            28  
[0]                                           <backward op>         0.01%       1.373ms         2.22%     499.017ms     249.508ms             2  
[0] autograd::engine::evaluate_function: LogSoftmaxBackw...         0.01%       1.328ms         0.15%      33.563ms      16.782ms             2  
[0]                                          StackBackward0         0.01%       1.265ms         0.03%       6.572ms     234.714us            28  
[0] autograd::engine::evaluate_function: torch::jit::(an...         0.00%     971.000us         2.22%     500.044ms     250.022ms             2  
[0]                                             aten::zero_         0.00%     967.000us         0.12%      28.103ms     121.

2023-03-20 02:46:21,206 - sigopt - INFO - Training completed based in sigopt suggestion, took 273.3797791004181 secs
2023-03-20 02:46:21,206 - E2EAIOK.SDA - INFO - training script completed



***    Best Trained Model    ***
  Model Type: rnnt
  Model Saved Path: 
  Sigopt Experiment id is None
  === Result Metrics ===
{'dataset_dir': '/home/vmagent/app/dataset/LibriSpeech', 'train_manifests': ['/home/vmagent/app/dataset/LibriSpeech/metadata/train-test.json'], 'val_manifests': ['/home/vmagent/app/dataset/LibriSpeech/metadata/dev-test.json']}

We found the best model! Here is the model explaination

***    Best Trained Model    ***
  Model Type: rnnt
  Model Saved Path: /home/vmagent/app/e2eaiok/result/357dc3f8a3dfe894b3a3fcdd15fd1129f95f71cf887c8475679b1ff5b50674d8
  Sigopt Experiment id is None
  === Result Metrics ===
    WER: 20.484347826086957
    training_time: 273.3797791004181
